In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/Users/chengpeng/Desktop/Research/STAT/CIBer')
import comonotonic as cm
import ensemble_ciber as ec
import math
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import copy
import utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import ensemble_ciber as ec
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTENC
import xlsxwriter

In [2]:
df = pd.read_csv('oil_spill.csv')
df.drop(df.columns[[0,22]], axis = 1, inplace=True)

In [3]:
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames

In [4]:
categorical = []
cont_col = [i for i in range(df.shape[1]-1)]
discrete_feature_val = None

In [5]:
#scaler = preprocessing.MinMaxScaler()
#scaler = preprocessing.PowerTransformer()
scaler = preprocessing.StandardScaler()
scale_col_name = ["X"+str(i) for i in cont_col]
df[scale_col_name] = scaler.fit_transform(df[scale_col_name])

In [6]:
x = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1].to_numpy()
smote = SMOTE(random_state=0)
x_resample, y_resample = smote.fit_resample(x, y)

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x_resample,y_resample,
                                                 test_size=0.2)

In [8]:
c_como_demo = cm.clustered_comonotonic(x_train,y_train,discrete_feature_val,cont_col,
                                      categorical, 0.995, None, corrtype='mutual_info',
                                      discrete_method='mdlp')
c_como_demo.run()

In [9]:
c_como_predict = c_como_demo.predict(x_test)
print(classification_report(y_test,c_como_predict))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       176
           1       0.98      0.99      0.99       183

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



In [11]:
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(x_train,y_train)
xgb_predict = xgb_clf.predict(x_test)
print(classification_report(y_test, xgb_predict))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       178
           1       0.98      1.00      0.99       181

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



In [12]:
rf_clf = RandomForestClassifier()
rf_clf.fit(x_train,y_train)
rf_predict = rf_clf.predict(x_test)
print(classification_report(y_test, rf_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       178
           1       0.99      0.99      0.99       181

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [13]:
ada_clf = AdaBoostClassifier()
ada_clf.fit(x_train,y_train)
ada_predict = ada_clf.predict(x_test)
print(classification_report(y_test, ada_predict))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97       178
           1       0.97      0.98      0.98       181

    accuracy                           0.97       359
   macro avg       0.98      0.97      0.97       359
weighted avg       0.98      0.97      0.97       359



In [15]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(x_train, y_train)
lgb_predict = lgb_clf.predict(x_test)
lgb_predict = lgb_predict.round(0).astype('int')
print(classification_report(y_test, lgb_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       178
           1       0.99      0.99      0.99       181

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



In [16]:
DT_clf = tree.DecisionTreeClassifier()
DT_clf.fit(x_train,y_train)
DT_predict = DT_clf.predict(x_test)
print(classification_report(y_test, DT_predict))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       178
           1       0.95      0.96      0.95       181

    accuracy                           0.95       359
   macro avg       0.95      0.95      0.95       359
weighted avg       0.95      0.95      0.95       359



In [17]:
svm = SVC()
svm.fit(x_train, y_train)
svm_predict = svm.predict(x_test)
print(classification_report(y_test, svm_predict))

/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.85      0.88      0.86       178
           1       0.87      0.85      0.86       181

    accuracy                           0.86       359
   macro avg       0.86      0.86      0.86       359
weighted avg       0.86      0.86      0.86       359



In [18]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)
print(classification_report(y_test, lr_predict))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       178
           1       0.90      0.88      0.89       181

    accuracy                           0.89       359
   macro avg       0.89      0.89      0.89       359
weighted avg       0.89      0.89      0.89       359



/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
ciber_nb = cm.clustered_comonotonic(x_train,y_train,discrete_feature_val,cont_col,
                                categorical, 1, None, corrtype='mutual_info',
                                discrete_method='mdlp')
ciber_nb.run()
ciber_nb_predict = ciber_nb.predict(x_test)
print(classification_report(y_test,ciber_nb_predict))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       170
           1       0.99      0.96      0.98       189

    accuracy                           0.97       359
   macro avg       0.97      0.98      0.97       359
weighted avg       0.98      0.97      0.97       359

